In [46]:
from __future__ import print_function, unicode_literals
import random
import logging
import os

os.environ['NLTK_DATA'] = os.getcwd() + '/nltk_data'

import spacy, spacy.symbols
import en_core_web_sm

nlp = en_core_web_sm.load()

logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Sentences we'll respond with if the user greeted us
GREETING_KEYWORDS = ("hello", "hi", "greetings", "sup", "what's up",)

GREETING_RESPONSES = ["I'm ready.", "Hey!", "*nods*", "Yep, I'm here!"]

FILTER_WORDS = ["com", "org", "io"]

# Sentences we'll respond with if we have no idea what the user just said
NONE_RESPONSES = [
    "I don't know what that means.",
    "Sir/Ma'am, this is a Wendy's.",
    "Could you rephrase that?",
]
# end

# If the user tries to tell us something about ourselves, use one of these responses
COMMENTS_ABOUT_SELF = [
    "I'm told that prettier robots get to work the register.",
    "But enough about me.",
]
# end


class UnacceptableUtteranceException(Exception):
    """Raise this (uncaught) exception if the response was going to trigger our blacklist"""
    pass

def bot_respond(sentence):
    """Main program loop: select a response for the input sentence and return it"""
    logger.info("OrderBot: respond to %s", sentence)
    resp = respond(sentence)
    return resp

def check_for_greeting(sentence):
    """If any of the words in the user's input was a greeting, return a greeting response"""
    for token in sentence:
        if token.text.lower() in GREETING_KEYWORDS:
            return random.choice(GREETING_RESPONSES)
        
def starts_with_vowel(word):
    """Check for pronoun compability -- 'a' vs. 'an'"""
    return True if word[0] in 'aeiou' else False

def find_pronoun(sent):
    """Given a sentence, find whether to respond in first or second person. Returns None if no candidate
    pronoun is found in the input"""
    pronoun = None

    for token in sent:
        # Disambiguate pronouns
        if token.text.lower() == 'you':
            pronoun = 'I'
        elif token.text == 'I':
            # If the user mentioned themselves, then they will definitely be the pronoun
            pronoun = 'You'
    return pronoun

def find_verb(sent):
    """Pick a candidate verb for the sentence."""
    verb = None
    pos = None
    for token in sent:
        if token.pos_ == "VERB":  # This is a verb
            verb = token
            pos = token.pos_
            break
    return token

def find_noun(sent):
    """Given a sentence, find the best candidate noun."""
    noun = None

    if not noun:
        for token in sent:
            if token.tag_.startswith('NN'):  # This is a noun
                noun = token.text
                break
    if noun:
        logger.info("Found noun: %s", noun)

    return noun

def find_adjective(sent):
    """Given a sentence, find the best candidate adjective."""
    adj = None
    for token in sent:
        if token.pos_ == 'ADJ':  # This is an adjective
            adj = token.text
            break
    return adj

def find_name(sent):
    """Given a sentence, find the best candidate name, or proper noun."""
    name = None

    if not name:
        for token in sent:
            if token.pos_ == 'PROPN':  # This is a name
                name = token.text
                break
    if name:
        logger.info("Found name: %s", name)

    return name

def construct_response(pronoun, noun, verb):
    """No special cases matched, so we're going to try to construct a full sentence that uses as much
    of the user's input as possible"""
    resp = []

    if pronoun:
        resp.append(pronoun)

    # We always respond in the present tense, and the pronoun will always either be a passthrough
    # from the user, or 'you' or 'I', in which case we might need to change the tense for some
    # irregular verbs.
    if verb:
        verb_word = verb[0]
        if verb_word in ('be', 'am', 'is', "'m"):  # This would be an excellent place to use lemmas!
            if pronoun.lower() == 'you':
                # The bot will always tell the person they aren't whatever they said they were
                resp.append("aren't really")
            else:
                resp.append(verb_word)
    if noun:
        pronoun = "an" if starts_with_vowel(noun) else "a"
        resp.append(pronoun + " " + noun)

    return " ".join(resp)

def acknowledge_task(name, task):
    """Issues an acknowledgement message that a task was logged for a particular person."""
    resp = random.choice(ACKNOWLEDGEMENT).format(**{'name': name, 'task': task})
    return resp

ACKNOWLEDGEMENT = [
    "Task logged for {name}: {task}",
]

def preprocess_text(sentence):
    """Handle some weird edge cases in parsing, like 'i' needing to be capitalized
    to be correctly identified as a pronoun"""
    cleaned = []
    words = sentence.split(' ')
    for w in words:
        if w == 'i':
            w = 'I'
        if w == "i'm":
            w = "I'm"
        cleaned.append(w)

    return ' '.join(cleaned)

# start:example-respond.py
def respond(sentence):
    """Parse the user's inbound sentence and find candidate terms that make up a best-fit response"""
    cleaned = preprocess_text(sentence)
    parsed = nlp(cleaned)

    # Loop through all the sentences, if more than one. This will help extract the most relevant
    # response text even across multiple sentences (for example if there was no obvious direct noun
    # in one sentence
    pronoun, noun, adjective, verb = find_candidate_parts_of_speech(parsed)
    name = find_name(parsed)

    # If we said something that sounds like a request or reminder, acknowledge
    resp = acknowledge_task(name, verb.text+" "+" ".join([t.text for t in verb.subtree]))

    # If we just greeted the bot, we'll use a return greeting
    if not resp:
        resp = check_for_greeting(parsed)

    if not resp:
        # If we didn't override the final sentence, try to construct a new one:
        if not pronoun:
            resp = random.choice(NONE_RESPONSES)
        elif pronoun == 'I' and not verb:
            resp = random.choice(COMMENTS_ABOUT_SELF)
        else:
            resp = construct_response(pronoun, noun, verb.text)

    # If we got through all that with nothing, use a random response
    if not resp:
        resp = random.choice(NONE_RESPONSES)

    logger.info("Returning phrase '%s'", resp)
    # Check that we're not going to say anything obviously offensive
    filter_response(resp)

    return resp

def find_candidate_parts_of_speech(parsed):
    """Given a parsed input, find the best pronoun, direct noun, adjective, and verb to match their input.
    Returns a tuple of pronoun, noun, adjective, verb any of which may be None if there was no good match"""
    pronoun = None
    noun = None
    adjective = None
    verb = None
    for sent in parsed.sents:
        pronoun = find_pronoun(sent)
        noun = find_noun(sent)
        adjective = find_adjective(sent)
        verb = find_verb(sent)
    logger.info("Pronoun=%s, noun=%s, adjective=%s, verb=%s", pronoun, noun, adjective, verb.text)
    return pronoun, noun, adjective, verb


# end

# start:example-filter.py
def filter_response(resp):
    """Don't allow any words to match our filter list"""
    tokenized = resp.split(' ')
    for word in tokenized:
        if '@' in word or '#' in word or '!' in word:
            raise UnacceptableUtteranceException()
        for s in FILTER_WORDS:
            if word.lower().startswith(s):
                raise UnacceptableUtteranceException()
# end

import sys
saying = "Hey Peter, while you're out get a burger."
print(bot_respond(saying))

INFO:root:OrderBot: respond to Hey Peter, while you're out get a burger.
INFO:root:Found noun: Peter
INFO:root:Pronoun=I, noun=Peter, adjective=None, verb=get
INFO:root:Found name: Peter
INFO:root:Returning phrase 'Task logged for Peter: get Hey Peter , while you 're out get a burger .'


Task logged for Peter: get Hey Peter , while you 're out get a burger .


In [45]:
#Isolated bot_respond call to play around with
bot_respond("I need Apple to stop removing audio jacks from things.")

INFO:root:OrderBot: respond to I need Apple to stop removing audio jacks from things.
INFO:root:Found noun: Apple
INFO:root:Pronoun=You, noun=Apple, adjective=audio, verb=need
INFO:root:Found name: Apple
INFO:root:Returning phrase 'Task logged for Apple: need I need Apple to stop removing audio jacks from things .'


'Task logged for Apple: need I need Apple to stop removing audio jacks from things .'